In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import time, os, pickle, pathlib

import model
import trainutils

report_every = 10  # how often to print stats during training
n_runs       = 1   # how many times to repeat the whole scan across beta's
savedirbase  = str(pathlib.Path().absolute()) + '/saveddata3/'

optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)

base_cfg = {
    'n_batch'         : 128 ,         # SGD batch size
    'train_noisevar'  : 'gradient',   # train noise variance with gradient descent ('gradient'), 
                                      #  scipy optimizer loop ('scipy'), or leave fixed ('none')
    'n_noisevar_batch': 1000,         # batch size for training noise variance when train_noisevar='scipy'
    'initial_fitvar'  : False,        # whether to set noisevar to optimal value before training
    'squaredIB'       : False,        # optimize I(Y;T)-beta*I(X;T) or I(Y;T)-beta*I(X;T)^2 
    'err_func'        : 'softmax_ce', # 'softmax_ce' for classification, 'mse' for regression  
}


runtype = 'MNIST'
#runtype = 'NoisyClassifier'
#runtype = 'Regression'
runtype = 'NoisyClassifierWine'

if runtype == 'MNIST':
    data = trainutils.load_mnist()
    savedir = runtype + '/v1'
    cfg = {
        'n_epochs'    : 150,
        'squaredIB'   : True,
        'encoder_arch': [(512,tf.nn.relu),(512,tf.nn.relu),(2,None)], 
        'decoder_arch': [(512,tf.nn.relu),(10,None)],
    }

elif runtype == 'NoisyClassifierWine':
    data = trainutils.load_wine()
    savedir = runtype + '/v10d'
    cfg = {
        'n_epochs'    : 300,
        'encoder_arch': [(10,tf.nn.relu),(10,tf.nn.relu),], 
        #'encoder_arch': [(10,tf.nn.relu),(10,tf.nn.relu),(3,tf.nn.relu),], 
        'decoder_arch': [(10,tf.nn.relu),(data['trn_Y'].shape[1],None)],
    }
    
elif runtype == 'NoisyClassifier':
    savedir = runtype + '/v1'
    data = trainutils.load_szt()
    cfg = {
        'n_epochs'      : 300,
        'encoder_arch'  : [(20,tf.nn.relu),(20,tf.nn.relu),(2,None)], 
        'decoder_arch'  : [(20,tf.nn.relu),(2,None)],
    }
    
elif runtype == 'Regression':
    savedir = runtype + '/v2' # v5sq-eta'
    # data generated by makeregressiondata.py
    with open('data/regression-100-10.pkl', 'rb') as f:
        data = pickle.load(f)
    labelcov = np.cov(data['trn_Y'].T)
    data['entropyY'] = 0.5 * np.log(np.linalg.det(2*np.pi*np.exp(1)*labelcov))
    
    cfg = {
        'n_epochs'         : 300,
        'encoder_arch'     : [(100,tf.nn.relu),(100,tf.nn.relu),(2,None)], 
        'decoder_arch'     : [(100,tf.nn.relu),(10,None)],
        'err_func'         : 'mse',
    }
    
else:
    raise Exception('unknown runtype')
    
savedir = savedirbase + savedir
for k, v in base_cfg.items():
    if k not in cfg: 
        cfg[k] = v
cfg['optimizer'] = repr(optimizer)

betavals = 10**np.linspace(-3, 1, 15, endpoint=True)


/home/artemy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()
sess=tf.Session()


n = model.Net(input_dims   = data['trn_X'].shape[1],
              encoder_arch = cfg['encoder_arch'], 
              decoder_arch = cfg['decoder_arch'],
              err_func     = cfg['err_func'],
              entropyY     = data['entropyY'],
              trainable_noisevar = cfg['train_noisevar']=='gradient', 
              noisevar     = 0.01)
saver = tf.train.Saver()


In [3]:
if True:
    print("Making base model")
    #cfg2['n_epochs'] = 10
    sess.run(tf.global_variables_initializer())

    trainutils.train(sess, 'ce', 0.0, cfg, data, n, optimizer, report_every, fname=savedir+'/results-base')

    save_path = saver.save(sess, savedir+'/_tf_basemodel')
    print("Model saved in path: %s" % save_path)

Making base model

mode: ce epoch: 1 | beta: 0.0000 | noisevar: 0.01 | kw: 0.00671535
ce:  0.947/ 0.929 | acc:  0.607/ 0.620 | loss:  0.947/ 0.929 | 
Ixt:  9.704/ 9.691 | Ixt_lb:  9.000/ 8.986 | vIxt:  883.845/ 887.745 | Iyt: -0.290/-0.272 | 

mode: ce epoch: 11 | beta: 0.0000 | noisevar: 0.01 | kw: 0.00671535
ce:  0.623/ 0.618 | acc:  0.631/ 0.637 | loss:  0.623/ 0.618 | 
Ixt:  9.751/ 9.738 | Ixt_lb:  9.084/ 9.065 | vIxt:  803.133/ 807.082 | Iyt:  0.034/ 0.039 | 

mode: ce epoch: 21 | beta: 0.0000 | noisevar: 0.01 | kw: 0.00671535
ce:  0.628/ 0.616 | acc:  0.623/ 0.637 | loss:  0.628/ 0.616 | 
Ixt:  9.717/ 9.716 | Ixt_lb:  9.023/ 9.014 | vIxt:  759.320/ 759.794 | Iyt:  0.030/ 0.041 | 

mode: ce epoch: 31 | beta: 0.0000 | noisevar: 0.01 | kw: 0.00671535
ce:  0.600/ 0.601 | acc:  0.642/ 0.647 | loss:  0.600/ 0.601 | 
Ixt:  9.735/ 9.750 | Ixt_lb:  9.027/ 9.043 | vIxt:  698.699/ 710.802 | Iyt:  0.057/ 0.056 | 

mode: ce epoch: 41 | beta: 0.0000 | noisevar: 0.01 | kw: 0.00671535
ce:  0.592

In [4]:
#sess  = tf.Session()
#n = model.Net(encoder_arch=[(512,'relu'),(512,'relu'),(2,'relu')], decoder_arch=[(512,'relu'),],
#              trainable_sigma=cfg['train_sigma'], log_sigma2=-2, log_eta2=-20, init_beta=0.0)
#loader = tf.train.import_meta_graph(basemodelpath+'.meta')
#saver.restore(sess, basemodelpath)


In [ ]:
for runndx in range(n_runs):
    for beta in betavals:
        if np.isclose(beta,0): 
            continue
        for mode in ['VIB','nlIB',]:
            saver.restore(sess, savedir+'/_tf_basemodel')
            sqmode = 'sq' if cfg['squaredIB'] else 'reg'
            fname = savedir + '/results-%s-%0.5f-%s-run%d' % (mode, beta, sqmode, runndx)
            print("Doing %s, beta=%0.4f, %s %s" % (mode, beta, sqmode, fname))
            trainutils.train(sess, mode, beta, cfg, data, n, optimizer, report_every=report_every, fname=fname, fit_var=cfg['initial_fitvar'])

            print()
            print()

Doing VIB, beta=0.0010, reg /home/artemy/nonlinearIB/code/saveddata3/NoisyClassifierWine/v10d/results-VIB-0.00100-reg-run0

mode: VIB epoch: 1 | beta: 0.0010 | noisevar: 0.01 | kw: 0.0276328
ce:  0.520/ 0.519 | acc:  0.739/ 0.747 | loss:  0.319/ 0.324 | 
Ixt:  13.453/ 13.481 | Ixt_lb:  12.297/ 12.313 | vIxt:  456.471/ 462.465 | Iyt:  0.137/ 0.138 | 

mode: VIB epoch: 11 | beta: 0.0010 | noisevar: 0.0126783 | kw: 0.00893617
ce:  0.517/ 0.524 | acc:  0.738/ 0.746 | loss: -0.055/-0.048 | 
Ixt:  9.041/ 9.034 | Ixt_lb:  7.666/ 7.636 | vIxt:  85.122/ 84.990 | Iyt:  0.141/ 0.133 | 

mode: VIB epoch: 21 | beta: 0.0010 | noisevar: 0.0159773 | kw: 0.00376706
ce:  0.527/ 0.515 | acc:  0.743/ 0.751 | loss: -0.077/-0.090 | 
Ixt:  6.910/ 6.901 | Ixt_lb:  5.434/ 5.422 | vIxt:  52.758/ 52.987 | Iyt:  0.130/ 0.143 | 

mode: VIB epoch: 31 | beta: 0.0010 | noisevar: 0.0196269 | kw: 0.00267327
ce:  0.529/ 0.524 | acc:  0.730/ 0.735 | loss: -0.085/-0.090 | 
Ixt:  5.991/ 5.980 | Ixt_lb:  4.423/ 4.426 | vIxt


mode: nlIB epoch: 41 | beta: 0.0010 | noisevar: 0.0185718 | kw: 0.0356095
ce:  0.535/ 0.530 | acc:  0.732/ 0.734 | loss: -0.110/-0.116 | 
Ixt:  11.928/ 11.926 | Ixt_lb:  10.611/ 10.619 | vIxt:  434.653/ 438.558 | Iyt:  0.122/ 0.128 | 

mode: nlIB epoch: 51 | beta: 0.0010 | noisevar: 0.0204135 | kw: 0.0241877
ce:  0.519/ 0.516 | acc:  0.730/ 0.743 | loss: -0.128/-0.131 | 
Ixt:  10.616/ 10.611 | Ixt_lb:  9.371/ 9.349 | vIxt:  431.271/ 424.268 | Iyt:  0.138/ 0.141 | 

mode: nlIB epoch: 61 | beta: 0.0010 | noisevar: 0.0216696 | kw: 0.0208815
ce:  0.524/ 0.523 | acc:  0.740/ 0.747 | loss: -0.124/-0.125 | 
Ixt:  10.134/ 10.121 | Ixt_lb:  8.898/ 8.891 | vIxt:  423.471/ 420.232 | Iyt:  0.134/ 0.135 | 

mode: nlIB epoch: 71 | beta: 0.0010 | noisevar: 0.0226502 | kw: 0.0361193
ce:  0.524/ 0.525 | acc:  0.735/ 0.740 | loss: -0.122/-0.121 | 
Ixt:  11.261/ 11.253 | Ixt_lb:  9.916/ 9.904 | vIxt:  415.119/ 415.262 | Iyt:  0.134/ 0.132 | 

mode: nlIB epoch: 81 | beta: 0.0010 | noisevar: 0.0239877 | k


mode: VIB epoch: 81 | beta: 0.0019 | noisevar: 0.0553348 | kw: 0.00157947
ce:  0.513/ 0.516 | acc:  0.746/ 0.738 | loss: -0.093/-0.090 | 
Ixt:  3.575/ 3.606 | Ixt_lb:  2.162/ 2.190 | vIxt:  26.556/ 26.566 | Iyt:  0.144/ 0.142 | 

mode: VIB epoch: 91 | beta: 0.0019 | noisevar: 0.06545 | kw: 0.00127913
ce:  0.512/ 0.521 | acc:  0.748/ 0.742 | loss: -0.098/-0.089 | 
Ixt:  3.238/ 3.238 | Ixt_lb:  1.903/ 1.899 | vIxt:  24.513/ 24.509 | Iyt:  0.145/ 0.136 | 

mode: VIB epoch: 101 | beta: 0.0019 | noisevar: 0.0777939 | kw: 0.00142116
ce:  0.512/ 0.522 | acc:  0.746/ 0.736 | loss: -0.101/-0.092 | 
Ixt:  3.084/ 3.084 | Ixt_lb:  1.806/ 1.796 | vIxt:  22.776/ 22.698 | Iyt:  0.145/ 0.136 | 

mode: VIB epoch: 111 | beta: 0.0019 | noisevar: 0.0915329 | kw: 0.00141794
ce:  0.515/ 0.524 | acc:  0.743/ 0.743 | loss: -0.101/-0.092 | 
Ixt:  2.901/ 2.922 | Ixt_lb:  1.680/ 1.689 | vIxt:  21.317/ 21.295 | Iyt:  0.142/ 0.133 | 

mode: VIB epoch: 121 | beta: 0.0019 | noisevar: 0.106363 | kw: 0.00138866
ce:  


mode: nlIB epoch: 131 | beta: 0.0019 | noisevar: 0.0415559 | kw: 0.0189404
ce:  0.513/ 0.514 | acc:  0.750/ 0.745 | loss: -0.128/-0.128 | 
Ixt:  8.333/ 8.331 | Ixt_lb:  6.990/ 6.975 | vIxt:  386.258/ 382.242 | Iyt:  0.144/ 0.144 | 

mode: nlIB epoch: 141 | beta: 0.0019 | noisevar: 0.0426229 | kw: 0.0288202
ce:  0.519/ 0.515 | acc:  0.744/ 0.740 | loss: -0.121/-0.125 | 
Ixt:  8.893/ 8.895 | Ixt_lb:  7.509/ 7.514 | vIxt:  376.132/ 375.171 | Iyt:  0.138/ 0.142 | 

mode: nlIB epoch: 151 | beta: 0.0019 | noisevar: 0.0430939 | kw: 0.0263594
ce:  0.513/ 0.521 | acc:  0.748/ 0.739 | loss: -0.127/-0.119 | 
Ixt:  8.719/ 8.710 | Ixt_lb:  7.346/ 7.339 | vIxt:  371.092/ 369.263 | Iyt:  0.144/ 0.136 | 

mode: nlIB epoch: 161 | beta: 0.0019 | noisevar: 0.0416248 | kw: 0.015693
ce:  0.515/ 0.508 | acc:  0.747/ 0.749 | loss: -0.127/-0.134 | 
Ixt:  8.084/ 8.081 | Ixt_lb:  6.746/ 6.751 | vIxt:  365.184/ 370.562 | Iyt:  0.142/ 0.149 | 

mode: nlIB epoch: 171 | beta: 0.0019 | noisevar: 0.041028 | kw: 0.02


mode: VIB epoch: 171 | beta: 0.0037 | noisevar: 0.298181 | kw: 0.00114483
ce:  0.533/ 0.533 | acc:  0.734/ 0.734 | loss: -0.086/-0.086 | 
Ixt:  1.879/ 1.875 | Ixt_lb:  0.986/ 0.982 | vIxt:  10.343/ 10.247 | Iyt:  0.124/ 0.124 | 

mode: VIB epoch: 181 | beta: 0.0037 | noisevar: 0.340563 | kw: 0.00125115
ce:  0.537/ 0.513 | acc:  0.723/ 0.743 | loss: -0.086/-0.110 | 
Ixt:  1.803/ 1.805 | Ixt_lb:  0.944/ 0.947 | vIxt:  9.349/ 9.375 | Iyt:  0.120/ 0.144 | 

mode: VIB epoch: 191 | beta: 0.0037 | noisevar: 0.386942 | kw: 0.0010011
ce:  0.534/ 0.527 | acc:  0.729/ 0.730 | loss: -0.092/-0.099 | 
Ixt:  1.706/ 1.719 | Ixt_lb:  0.887/ 0.902 | vIxt:  8.306/ 8.410 | Iyt:  0.123/ 0.131 | 

mode: VIB epoch: 201 | beta: 0.0037 | noisevar: 0.429631 | kw: 0.000963343
ce:  0.524/ 0.534 | acc:  0.730/ 0.729 | loss: -0.105/-0.095 | 
Ixt:  1.668/ 1.652 | Ixt_lb:  0.876/ 0.859 | vIxt:  7.546/ 7.428 | Iyt:  0.133/ 0.123 | 

mode: VIB epoch: 211 | beta: 0.0037 | noisevar: 0.470083 | kw: 0.00086437
ce:  0.545/

...


mode: VIB epoch: 221 | beta: 1.3895 | noisevar: 0.8131 | kw: 6.31807e-06
ce:  0.660/ 0.660 | acc:  0.633/ 0.629 | loss:  0.524/ 0.524 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  0.375/ 0.375 | Iyt: -0.003/-0.003 | 

mode: VIB epoch: 231 | beta: 1.3895 | noisevar: 0.867202 | kw: 2.50339e-06
ce:  0.662/ 0.660 | acc:  0.624/ 0.628 | loss:  0.262/ 0.260 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  0.185/ 0.185 | Iyt: -0.005/-0.003 | 

mode: VIB epoch: 241 | beta: 1.3895 | noisevar: 0.910878 | kw: 2.50339e-06
ce:  0.661/ 0.659 | acc:  0.627/ 0.631 | loss:  0.117/ 0.116 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  0.082/ 0.082 | Iyt: -0.004/-0.002 | 

mode: VIB epoch: 251 | beta: 1.3895 | noisevar: 0.943908 | kw: 3.57627e-06
ce:  0.662/ 0.659 | acc:  0.624/ 0.630 | loss:  0.049/ 0.047 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  0.032/ 0.032 | Iyt: -0.005/-0.002 | 

mode: VIB epoch: 261 | beta: 1.3895 | noisevar: 0.967148 | kw: 1.90735e-06
ce:  0.659


mode: nlIB epoch: 261 | beta: 1.3895 | noisevar: 0.0134508 | kw: 1.22761e-13
ce:  0.662/ 0.656 | acc:  0.624/ 0.636 | loss:  0.004/-0.001 | 
Ixt: -0.000/-0.000 | Ixt_lb: -0.000/-0.000 | vIxt:  38.089/ 38.089 | Iyt: -0.004/ 0.001 | 

mode: nlIB epoch: 271 | beta: 1.3895 | noisevar: 0.0134412 | kw: 1.22761e-13
ce:  0.662/ 0.663 | acc:  0.626/ 0.623 | loss:  0.004/ 0.005 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  38.096/ 38.096 | Iyt: -0.004/-0.005 | 

mode: nlIB epoch: 281 | beta: 1.3895 | noisevar: 0.013436 | kw: 1.22761e-13
ce:  0.657/ 0.662 | acc:  0.634/ 0.624 | loss: -0.000/ 0.004 | 
Ixt: -0.000/-0.000 | Ixt_lb: -0.000/-0.000 | vIxt:  38.099/ 38.099 | Iyt:  0.000/-0.004 | 

mode: nlIB epoch: 291 | beta: 1.3895 | noisevar: 0.0134298 | kw: 1.22761e-13
ce:  0.658/ 0.667 | acc:  0.632/ 0.615 | loss:  0.001/ 0.009 | 
Ixt: -0.000/-0.000 | Ixt_lb: -0.000/-0.000 | vIxt:  38.104/ 38.104 | Iyt: -0.001/-0.009 | 

mode: nlIB epoch: 301 | beta: 1.3895 | noisevar: 0.0134254 | kw: 1.2


mode: VIB epoch: 301 | beta: 2.6827 | noisevar: 0.998433 | kw: 4.76836e-06
ce:  0.658/ 0.662 | acc:  0.633/ 0.623 | loss:  0.000/ 0.005 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  0.000/ 0.000 | Iyt: -0.000/-0.005 | 


Doing nlIB, beta=2.6827, reg /home/artemy/nonlinearIB/code/saveddata3/NoisyClassifierWine/v10d/results-nlIB-2.68270-reg-run0

mode: nlIB epoch: 1 | beta: 2.6827 | noisevar: 0.01 | kw: 0.0265005
ce:  0.520/ 0.517 | acc:  0.739/ 0.742 | loss:  35.639/ 35.650 | 
Ixt:  13.336/ 13.341 | Ixt_lb:  12.160/ 12.177 | vIxt:  465.653/ 457.132 | Iyt:  0.137/ 0.140 | 

mode: nlIB epoch: 11 | beta: 2.6827 | noisevar: 0.0120403 | kw: 2.37224e-05
ce:  0.804/ 0.815 | acc:  0.372/ 0.361 | loss:  4.101/ 4.051 | 
Ixt:  1.474/ 1.452 | Ixt_lb:  0.770/ 0.752 | vIxt:  39.825/ 39.823 | Iyt: -0.147/-0.157 | 

mode: nlIB epoch: 21 | beta: 2.6827 | noisevar: 0.0121522 | kw: 7.62937e-06
ce:  0.664/ 0.664 | acc:  0.627/ 0.624 | loss:  0.015/ 0.020 | 
Ixt:  0.003/ 0.005 | Ixt_lb:  0.003/ 0.


mode: VIB epoch: 31 | beta: 5.1795 | noisevar: 0.0210339 | kw: 3.00403e-05
ce:  0.659/ 0.664 | acc:  0.625/ 0.623 | loss:  174.171/ 174.186 | 
Ixt:  0.375/ 0.429 | Ixt_lb:  0.145/ 0.171 | vIxt:  33.627/ 33.629 | Iyt: -0.002/-0.007 | 

mode: VIB epoch: 41 | beta: 5.1795 | noisevar: 0.0269045 | kw: 2.43184e-05
ce:  0.666/ 0.660 | acc:  0.617/ 0.633 | loss:  161.414/ 161.407 | 
Ixt:  0.185/ 0.174 | Ixt_lb:  0.067/ 0.064 | vIxt:  31.163/ 31.162 | Iyt: -0.008/-0.003 | 

mode: VIB epoch: 51 | beta: 5.1795 | noisevar: 0.0343703 | kw: 2.4676e-05
ce:  0.665/ 0.671 | acc:  0.618/ 0.618 | loss:  148.733/ 148.739 | 
Ixt:  0.111/ 0.106 | Ixt_lb:  0.038/ 0.037 | vIxt:  28.714/ 28.714 | Iyt: -0.008/-0.014 | 

mode: VIB epoch: 61 | beta: 5.1795 | noisevar: 0.043837 | kw: 4.17232e-06
ce:  0.657/ 0.658 | acc:  0.637/ 0.631 | loss:  136.139/ 136.140 | 
Ixt:  0.063/ 0.065 | Ixt_lb:  0.020/ 0.020 | vIxt:  26.284/ 26.284 | Iyt:  0.000/-0.001 | 

mode: VIB epoch: 71 | beta: 5.1795 | noisevar: 0.0557973 | kw


mode: nlIB epoch: 71 | beta: 5.1795 | noisevar: 0.011764 | kw: 3.64165e-08
ce:  0.662/ 0.655 | acc:  0.625/ 0.640 | loss:  0.004/-0.002 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  39.434/ 39.434 | Iyt: -0.004/ 0.003 | 

mode: nlIB epoch: 81 | beta: 5.1795 | noisevar: 0.0117637 | kw: 2.47145e-08
ce:  0.656/ 0.657 | acc:  0.635/ 0.635 | loss: -0.002/-0.001 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  39.435/ 39.435 | Iyt:  0.002/ 0.001 | 

mode: nlIB epoch: 91 | beta: 5.1795 | noisevar: 0.0117635 | kw: 1.19237e-08
ce:  0.665/ 0.662 | acc:  0.617/ 0.624 | loss:  0.008/ 0.005 | 
Ixt:  0.000/ 0.000 | Ixt_lb:  0.000/ 0.000 | vIxt:  39.434/ 39.434 | Iyt: -0.008/-0.005 | 


In [ ]:
# Code to plot activations
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
if False:
    plt.figure(figsize=(10,10))
    x, y = data['tst_X'], data['tst_Y']
    mx = sess.run(n.encoder[-1], feed_dict={n.x: x})
    var = sess.run(n.noisevar)
    ax = plt.axes()
    for r in mx:
        c = plt.Circle((r[0], r[1]), radius=np.sqrt(var), fc='none', alpha=0.05, ec='k')
        ax.add_patch(c)
    plt.axis('scaled');